# IE 306.02 System Simulation Homework 1

**Berkay Bör - Ege Can Kaya - Zuhal Didem Aytaç**

**2017400138 - 2018400018 - 2018400045**

This is our simulation for IE 306.02 System Simulation Homework 1.

In this discrete event simulation, $  N = 1201  $ individuals get sick at random times with an exponantial rate $ \lambda = 1/300 $. An individual who is sick either goes to the hospital with probability 0.2 or stays at home with probability  $  0.8 $ . There are $ K = 51 $ beds in the hospital. If all K beds are full when an individual arrives to the hospital, that patient is rejected and treated home. 


In [ ]:
# outputs are too long, especially for simulations with 100,000 time units
# google colab only shows the last 5000 lines of the output
# therefore, to get to full output, we need to write the output to a file
# we chose to write to a file on google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import the requirements
!pip install simpy
import os
import sys
import random
import simpy

# the below block is needed to write to a file on google drive

os.chdir("/content/drive/")
out = open('/content/drive/MyDrive/ie306-p1/output-1.txt','w')


The input variables are calculated in the following way:

 $ S = 045 + 018 + 138 = 201  $ 
 
 $ N = 1000+S=1201  $ 

 $  K  = ceiling(N/24) = 51  [beds]  $ 

getting sick rate  $  LAMBDA =  \lambda  = 1/300  [patients/day]  $ 


A sick person is healed with an independent exponential time with following rates:

 $ RATE_1 = 1/6 [days]^{-1} $ (healing rate for a patient being treated in the hospital)

 $  RATE_2 = 1/10 [days]^{-1}$ (healing rate for a patient that does not go to the hospital) 

When an individual is rejected due to full capacity and is home treated, the healing time is longer, with r ~ U[1,2] 

 $  RATE_3 = RATE_1 * r [days]^{-1}$ (healing rate for a patient that is rejected by the hospital due to full capacity)


In [ ]:
# define the input variables
N = 1201
K = 51
LAMBDA = 1/300
RATE_1 = 1/6
RATE_2 = 1/10
RANDOM_SEED = 1
SIM_TIME = 1000
NO_INITIAL_ILL = 0
random.seed(RANDOM_SEED)

We define the necessary set of arrays and global variables

In [ ]:
# we need event based statistics, so when an event occurs, we log the current state of the system

no_sick = 0       # number of sick people
no_hospital = 0   # number of people getting treatment in the hospital (number of full beds)
no_healed = 0     # number of healed people

no_events = 0     # number of events occured
no_empty = 0      # number of event that the beds are all empty

sickness_times = []
occupied_beds = []  # event based list of number of occupied beds in the hospital
sick_people = []    # event based list of proportion of sick people in the population

The get sick in random with exponential rate (interarrival times)

In [ ]:
def time_to_ailment():
  return random.expovariate(LAMBDA)

The class definition for the people arriving at the modeled system.

Once a person gets sick, he/she immediately goes to the hospital with probability 0.2 <br>
If so, and if the hospital is not full, the person is assigned to a bed and treated in the hospital<br>
If the hospital is full, the person is treated at home. <br>
If the person does not go to the hospital, he/she immediately starts home remedies.

The processes for hospital treatment, home treathment and home remedies are defined according to the description with individual rates defined above. 

Details can be found in the comments

In [ ]:
class Person(object):
  def __init__(self, env, name, hospital, dummy):
    self.env = env
    self.name = name
    self.dummy = dummy
    self.ill = False

    if not dummy:
      self.process = env.process(self.living(hospital))
      env.process(self.ail())
    else:
      self.process = env.process(self.direct_hospitalization(hospital))

  # this part implements the cases where we start with the hospital half full and full
  # in these cases, we start with having K/2 or K beds occupied at time 0.
  def direct_hospitalization(self, hospital):
    global no_sick, no_hospital, no_healed
    self.ill = True
    no_sick += 1

    with hospital.request() as req:
      yield req
      no_hospital += 1
      out.write('%s got assigned to a bed at hospital, time: 0, no. of sick people: %g, no. of people in hospital: %g\n' 
      % (self.name, no_sick, no_hospital))

      yield self.env.process(self.get_treatment())
      self.ill = False
      no_hospital -= 1
      no_sick -= 1
      no_healed += 1
      out.write('%s healed, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
      % (self.name, self.env.now, no_sick, no_hospital))


      self.process = env.process(self.living(hospital))
      env.process(self.ail())
  
  # this part implements the main body of the simulation
  def living(self, hospital):
    global no_sick, no_hospital, no_events, no_empty, no_healed
    while True:
      # until the simulation time ends, people getting ill raise interrupts to the system
      try:
        yield env.timeout(SIM_TIME)
      
      # a person became ill and raise an interrupt to the system.
      except simpy.Interrupt:
        self.ill = True
        no_sick += 1
        prob = random.uniform(0, 1)

        # An individual who is sick goes to the hospital with probability 0.2 
        if prob < .2:

          # if the hospital is not full, a bed is requested
          if no_hospital != K:
            with hospital.request() as req:
              yield req
              no_hospital += 1
              out.write('%s got assigned to a bed at hospital, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
              % (self.name, self.env.now, no_sick, no_hospital))
              no_events += 1
              occupied_beds.append(no_hospital)
              sick_people.append(no_sick/N)
              yield self.env.process(self.get_treatment())

              # After the treatment process ends, the person is no longer ill
              # We make the necessary changes in variables
              # We log this event and the current state of the system
              self.ill = False
              no_hospital -= 1
              no_sick -= 1
              no_healed += 1
              out.write('%s healed, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
              % (self.name, self.env.now, no_sick, no_hospital))
              no_events += 1
              if no_hospital == 0: no_empty += 1
              occupied_beds.append(no_hospital)
              sick_people.append(no_sick/N)

          # if the hospital is full, the person is rejected and starts home treatment
          else:
            out.write('%s got rejected from hospital and started home treatment, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
            % (self.name, self.env.now, no_sick, no_hospital))
            no_events += 1
            if no_hospital == 0: no_empty += 1
            occupied_beds.append(no_hospital)
            sick_people.append(no_sick/N)
            yield self.env.process(self.rejected_treatment())

            # After the treatment process ends, the person is no longer ill
            # We make the necessary changes in variables
            # We log this event and the current state of the system
            self.ill = False
            no_sick -= 1
            no_healed += 1
            out.write('%s healed, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
            % (self.name, self.env.now, no_sick, no_hospital))

            no_events += 1
            if no_hospital == 0: no_empty += 1
            occupied_beds.append(no_hospital)
            sick_people.append(no_sick/N)
          
        # An individual who is sick stays at home with probability  0.8
        else:
          out.write('%s started home treament, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
          % (self.name, self.env.now, no_sick, no_hospital))
          no_events += 1
          if no_hospital == 0: no_empty += 1
          occupied_beds.append(no_hospital)
          sick_people.append(no_sick/N)
          yield self.env.process(self.self_care())

          # After the treatment process ends, the person is no longer ill
          # We make the necessary changes in variables
          # We log this event and the current state of the system
          self.ill = False
          no_sick -= 1
          no_healed += 1
          out.write('%s healed, time: %g, no. of sick people: %g, no. of people in hospital: %g\n' 
          % (self.name, self.env.now, no_sick, no_hospital))
          no_events += 1
          if no_hospital == 0: no_empty += 1
          occupied_beds.append(no_hospital)
          sick_people.append(no_sick/N)

# this function implements the process of a person becoming ill
# the time people become ill is random and exponential with rate lambda  
  def ail(self):
    while True:
      yield self.env.timeout(time_to_ailment())
      if not self.ill:
        self.process.interrupt()

# this function implements the case when a person is treated in the hospital
# the healing process is random and exponential with rate RATE_1 
# after the healing duration ends, this function returns and necessary actions are taken
  def get_treatment(self):
    duration = random.expovariate(RATE_1)
    yield self.env.timeout(duration)
    sickness_times.append(duration)

# this function implements the case when a person does not go to the hospital and is treated with self care
# the healing process is random and exponential with rate RATE_2
# after the healing duration ends, this function returns and necessary actions are taken
  def self_care(self):
    duration = random.expovariate(RATE_2)
    yield self.env.timeout(duration)
    sickness_times.append(duration)

# this function implements the case when a person goes to the hospital and is rejected due to full capacity
# the healing process takes longer than it would normally take in the hospital
# the healing process is random and exponential with rate RATE_2
# after the healing duration ends, this function returns and necessary actions are taken
  def rejected_treatment(self):
    r = random.uniform(1, 2)
    duration = random.expovariate((1/r)*RATE_1)
    yield self.env.timeout(duration)
    sickness_times.append(duration)

Define the environment, population and run the simulation

In [ ]:
env = simpy.Environment()
hospital = simpy.Resource(env, capacity=K)
population = [Person(env, 'Person %d' % i, hospital, i < NO_INITIAL_ILL)
            for i in range(N)]  

env.run(SIM_TIME)


Print out the model responses

In [ ]:
out.write('Long run probability of hospital being empty: %g\n' % (no_empty/no_events))
out.write('Average number of occupied beds: %g\n' % (sum(occupied_beds)/len(occupied_beds)))
out.write('Average proportion of sick population: %g\n' % (sum(sick_people)/len(sick_people)))
out.write('Total average sickness time: %g\n' % (sum(sickness_times)/len(sickness_times)))
out.write('Random number seed: %g\n' % RANDOM_SEED)
out.write('Simulation length: %g\n' % SIM_TIME)
out.write('No. of initially occupied beds: %g' % NO_INITIAL_ILL)

out.close()